In [6]:
%pylab inline
import pandas as pd
import os 

Populating the interactive namespace from numpy and matplotlib


Construcción base de datos:

In [7]:
#Inicializo las columnas con los nombres de las localidades que ya tienen datos
localidades=os.listdir("Votaciones_localidades")
Nombreslocalidades=[]
for i in localidades:
    Nombreslocalidades.append(i[:-7])
df = pd.DataFrame(columns=[Nombreslocalidades])
df= df.rename_axis(None)


In [8]:
#Algoritmo que ingresa los datos a la tabla, la tabla solo contiene las localidades reportadas.
Index=('P_Liberal','P_Conservador','P_Opcion_Ciudadana','P_Cambio_Radical','P_Alianza_Verde','P_MIRA','P_de_la_U','Polo','P_Centro_Democratico','P_SOMOS','FARC','Colombia_Justa_Libres','SI_SE_PUEDE',"Union_con_Fortaleza","Todos_somos_Colombia",'Coalicion_Decencia','Voto_Blanco_Nacional','AICO','ASI','MAIS','SOBERANIA','MIA','RenovacionEtnica','CASIYOUREN','Voto_Blanco_Indigena','Nulos','No_Marcados',"Potencial votantes", "Referencia")
for i in localidades:
    temp=pd.read_csv("Votaciones_localidades/"+str(i))
    temp.index = Index
    df[str(i[:-7])] = temp.iloc[:-1,1].apply(pd.to_numeric)

## Base de datos completa

In [9]:
#Suma el total de cada Item en Bogotá y genera un archivo CSV con la información
dc=df.copy()

dc["Total Bogota"]=dc.sum(axis=1)
dc.to_csv("../Bogota/Votacion_Bogota_Unificada.csv")

## Circunscripcion nacional

In [10]:
#Suma el total de cada Item en Bogotá y genera un archivo CSV con la información de votos en circunscripcion nacional
dfn=df[:-11].copy()
dfn["Total Bogota"]=dfn.sum(axis=1)
dfn.to_csv("../Bogota/Circunscripcion_Nacional_Bogota.csv")

## Circunscripcion Indigena

In [11]:
#Suma el total de cada Item en Bogotá y genera un archivo CSV con la información de votos en circunscripcion indigena
dfi=df[-11:-3].copy()
dfi["Total Bogota"]=dfi.sum(axis=1)
dfi.to_csv("../Bogota/Circunscripcion_Indigena_Bogota.csv")

## Datos Participacion


In [12]:
#calcula los votantes validos en circunscripcion nacional
Votantes_nacional=dfn.sum(axis=0).copy()
#calcula los votantes validos en circunscripcion indigena
Votantes_indigena=dfi.sum(axis=0).copy()
#Copia votos nulos y no marcados
dfp=df[-3:-1].copy()

#Agrega los datos de votantes validos
dfp["Total Bogota"]=dfp.sum(axis=1)
Votantes_nacional.name='Votos validos Circuns. Nacional'
Votantes_indigena.name='Votos validos Circuns. Indigena'
dfp=dfp.append([Votantes_indigena,Votantes_nacional]).copy()

#Con todos los datos suma para determinar numero de votantes, es decir votos validos, nulos y no marcados sumados.
Votantes=dfp.sum(axis=0)
Votantes.name='Total Votantes*Validos+Nulos+No Marcados*'

#Agrega datos de potencial de votantes, es decir numero de ciudadanos habilitados para votar
potencial=dc.loc["Potencial votantes"].copy()
dfp=dfp.append([Votantes,potencial]).copy()
Porcen_votantes=((100*dfp.loc["Total Votantes*Validos+Nulos+No Marcados*"]/dfp.loc["Potencial votantes"])).copy()

##formato para 2 cifras significativas en porcentaje votantes
Porcen_votantes.name="Porcentaje votantes(%)"
dfp=dfp.append(Porcen_votantes).copy()
formato= lambda x: format(x,'.2f')
dfp.loc["Porcentaje votantes(%)"]=dfp.loc['Porcentaje votantes(%)'].apply(formato)
dfp.to_csv("../Bogota/Participacion_bogota.csv")